In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from model import ResNet20
from Train import model_training

In [2]:
# Hyper parameters
epoch_losses = []
train_loss_list = []
epochs = 50
lr = 0.01
batchsize = 64

# Model device assignment
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNet20().to(device)

In [ ]:
# Define transformation
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

# Load the training dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

# Load the test dataset
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

# Define the class names for reference
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# Testing model on Cifar10
testing_module = model_training(model, lr, trainloader, epochs)
train_loss = testing_module.forward(device)

Start Training for 50 epochs
training cycle: 0
Epoch 0, Step 1, Loss: 2.6719
Epoch 0, Step 1001, Loss: 2.0836
Epoch 0, Step 2001, Loss: 2.2205
Epoch 0, Step 3001, Loss: 2.3944
Epoch 0, Step 4001, Loss: 2.2846
Epoch 0, Step 5001, Loss: 1.9003
Epoch 0, Step 6001, Loss: 2.0460
Epoch 0, Step 7001, Loss: 2.2733
Epoch 0, Step 8001, Loss: 2.3869
Epoch 0, Step 9001, Loss: 2.3339
Epoch 0, Step 10001, Loss: 2.3199
Epoch 0, Step 11001, Loss: 2.2967
Epoch 0, Step 12001, Loss: 2.3461
Epoch [1/50], Average Loss: 2.2494
training cycle: 1
Epoch 1, Step 1, Loss: 2.2723
Epoch 1, Step 1001, Loss: 2.3048
Epoch 1, Step 2001, Loss: 2.3122
Epoch 1, Step 3001, Loss: 2.3403
Epoch 1, Step 4001, Loss: 2.3274
Epoch 1, Step 5001, Loss: 2.3027
Epoch 1, Step 6001, Loss: 2.3176
Epoch 1, Step 7001, Loss: 2.2482
Epoch 1, Step 8001, Loss: 2.2997
Epoch 1, Step 9001, Loss: 2.2803
Epoch 1, Step 10001, Loss: 2.2726
Epoch 1, Step 11001, Loss: 2.3209
Epoch 1, Step 12001, Loss: 2.3391
Epoch [2/50], Average Loss: 2.3075
trainin

In [ ]:
# Visualization of training results 
plt.plot(train_loss, epochs, linewidth=2, c='r')
plt.xlabel("Epochs")
plt.ylabel("Training Loss")
plt.title("Training Loss of ResNet20 Model")
plt.grid(True)

plt.tight_layout()
plt.show()